In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [2]:
permits = pd.read_csv('./datasets/building_permits_2017.csv')
permits.head()

,permit_id,issued,cost,lat,lng
0,2017032777,2017-05-24,226201.0,36.198241,-86.742235
1,2017061156,2017-10-23,15000.0,36.151554,-86.830222
2,2017074521,2017-11-20,13389.0,36.034239,-86.708892
3,2017035041,2017-05-30,40000.0,36.128659,-86.841815
4,2017000049,2017-01-09,268107.0,36.045042,-86.616211


In [3]:
council_districts = gpd.read_file('./datasets/council_districts.geojson')[['district','geometry']]
council_districts.head()

,district,geometry
0,1,(POLYGON ((-86.90738248774342 36.3905151283193...
1,2,(POLYGON ((-86.75902399986667 36.2309080000732...
2,8,(POLYGON ((-86.72850199989709 36.2832840002146...
3,9,(POLYGON ((-86.68680500011935 36.2867050001350...
4,4,(POLYGON ((-86.74488864807594 36.0531632050230...


## Finding counts from a spatial join

In [4]:
# Create a shapely Point from lat and lng
permits['geometry'] = permits.apply(lambda x: Point((x.lng , x.lat)), axis = 1)

# Build a GeoDataFrame: permits_geo
permits_geo = gpd.GeoDataFrame(permits, crs = council_districts.crs, geometry = permits.geometry)

# Spatial join of permits_geo and council_districts
permits_by_district = gpd.sjoin(permits_geo, council_districts, op = 'within')
permits_by_district.head(2)

,permit_id,issued,cost,lat,lng,geometry,index_right,district
0,2017032777,2017-05-24,226201.0,36.198241,-86.742235,POINT (-86.74223499999999 36.198241),5,5
68,2017053890,2017-09-05,0.0,36.185442,-86.768239,POINT (-86.76823900000001 36.185442),5,5


In [5]:
# Create permit_counts
permit_counts = permits_by_district.groupby(['district']).size()
permit_counts.head()

district
1     146
10    119
11    239
12    163
13    139
dtype: int64

## Council district areas and permit counts

In [6]:
# Create an area column in council_districts
council_districts['area'] = council_districts.geometry.area
council_districts.head(2)

,district,geometry,area
0,1,(POLYGON ((-86.90738248774342 36.3905151283193...,0.022786
1,2,(POLYGON ((-86.75902399986667 36.2309080000732...,0.002927


In [7]:
# Convert permit_counts to a DataFrame
permits_df = permit_counts.to_frame()
print(permits_df.head(2))

            0
district     
1         146
10        119


In [8]:
# Reset index and column names
permits_df.reset_index(inplace=True)
permits_df.columns = ['district', 'bldg_permits']
print(permits_df.head(2))

  district  bldg_permits
0        1           146
1       10           119


In [9]:
# Merge council_districts and permits_df: 
districts_and_permits = pd.merge(council_districts, permits_df, on = 'district')
districts_and_permits.head(2)

,district,geometry,area,bldg_permits
0,1,(POLYGON ((-86.90738248774342 36.3905151283193...,0.022786,146
1,2,(POLYGON ((-86.75902399986667 36.2309080000732...,0.002927,399


## Calculating a normalized metric

In [11]:
# Print the type of districts_and_permits and change the area of unit to cm from degree
print(type(districts_and_permits))
districts_and_permits['area'] = districts_and_permits['geometry'].to_crs(epsg = 3857).area / (10**6)
districts_and_permits.head()

<class 'geopandas.geodataframe.GeoDataFrame'>


,district,geometry,area,bldg_permits
0,1,(POLYGON ((-86.90738248774342 36.3905151283193...,350.194851,146
1,2,(POLYGON ((-86.75902399986667 36.2309080000732...,44.956987,399
2,8,(POLYGON ((-86.72850199989709 36.2832840002146...,38.667932,209
3,9,(POLYGON ((-86.68680500011935 36.2867050001350...,44.295293,186
4,4,(POLYGON ((-86.74488864807594 36.0531632050230...,31.441618,139


In [13]:
# Create permit_density column in districts_and_permits
districts_and_permits['permit_density'] = districts_and_permits.apply(lambda row: row.bldg_permits / row.area, axis = 1)

# Print the head of districts_and_permits
districts_and_permits.head()

,district,geometry,area,bldg_permits,permit_density
0,1,(POLYGON ((-86.90738248774342 36.3905151283193...,350.194851,146,0.416911
1,2,(POLYGON ((-86.75902399986667 36.2309080000732...,44.956987,399,8.875150
2,8,(POLYGON ((-86.72850199989709 36.2832840002146...,38.667932,209,5.404995
3,9,(POLYGON ((-86.68680500011935 36.2867050001350...,44.295293,186,4.199092
4,4,(POLYGON ((-86.74488864807594 36.0531632050230...,31.441618,139,4.420892
